<a href="https://colab.research.google.com/github/wolfinwallst/All-About-Financial-Data-Science/blob/main/NLP_RNN_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[딥 러닝 파이토치 교과서](https://wikidocs.net/60690) 를 참고했다.
먼저 PyTorch를 사용하지 않고, RNN 구조를 다뤄보자:

In [1]:
import numpy as np

timesteps = 10 # 시점 개수, NLP에선 문장 길이

input_size = 4 # 입력 차원. NLP에선 단어 벡터의 차원
hidden_size = 8 # 은닉 상태의 크기, 메모리 셀의 용량

inputs = np.random.random((timesteps, input_size)) # 입력에 해당되는 2D 텐서
hidden_state_t = np.zeros((hidden_size,)) # 은닉 상태 0으로 초기화

In [2]:
print(hidden_state_t)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [3]:
print(inputs.shape)

(10, 4)


$$ h_t = \tanh ( W_x X_t + W_h h_{t-1} + b) $$
$$ y_t = f (W_y h_t + b) $$

여기서 f는 non-linear activation function으로, 대표적으로 sigmoid를 사용 할 수 있다.

In [4]:
# 파라미터(가중치, 편향) 초기화
Wx = np.random.random((hidden_size, input_size)) # (8, 4) 크기의 2D 텐서
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)
b = np.random.random((hidden_size,)) # (8,) 1D 텐서

In [5]:
print(np.shape(Wx))
print(Wh.shape)
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


In [6]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs:
    output_t = np.tanh(Wx @ input_t + Wh @ hidden_state_t + b)
    total_hidden_states.append(list(output_t))

    print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
    hidden_state_t = output_t

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)


In [7]:
total_hidden_states = np.stack(total_hidden_states, axis=0)
print(total_hidden_states)

[[0.98434029 0.96913257 0.96700017 0.96173097 0.85118625 0.93472664
  0.9634645  0.95376834]
 [0.99996726 0.99998777 0.99964854 0.9999945  0.99939482 0.99992895
  0.99997089 0.99962376]
 [0.99999623 0.99999788 0.99987541 0.99999794 0.99969308 0.99997113
  0.99998785 0.99995163]
 [0.99999678 0.99999839 0.99993058 0.99999906 0.99984022 0.99998921
  0.99999195 0.99995456]
 [0.99998656 0.99999067 0.99966201 0.99999467 0.99921138 0.99991535
  0.99997474 0.99975024]
 [0.99999465 0.99999828 0.99990452 0.9999986  0.99981838 0.99998024
  0.99998855 0.99996574]
 [0.99998921 0.99999191 0.99971958 0.99999574 0.99933731 0.99993743
  0.99997776 0.99977848]
 [0.99999087 0.99999044 0.99971211 0.99999582 0.99929755 0.99994443
  0.99997636 0.99974082]
 [0.99999856 0.99999915 0.99997243 0.99999967 0.99991698 0.99999722
  0.9999969  0.99995056]
 [0.99999066 0.99999731 0.9998864  0.99999833 0.9997416  0.99997483
  0.99999043 0.99989675]]


# 2. PyTorch 로 `RNN` 을 다뤄본다:

In [8]:
import torch
import torch.nn as nn

In [9]:
input_size = 5
hidden_size = 8 # RNN의 대표적인 하이퍼파라미터

# 입력 텐서 정의, (batch_size, 시점 개수, 매 시점마다 들어가는 입력 개수)
inputs = torch.Tensor(1, 10, 5)

In [10]:
# RNN의 셀 생성
# batch_first=True -> 입력 텐서의 첫번째 차원이 배치 크기이다.
cell = nn.RNN(input_size, hidden_size, batch_first=True)

outputs, _status = cell(inputs)
print(outputs)
print(outputs.shape) # 모든 time-step의 hidden_state

tensor([[[nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan]]],
       grad_fn=<TransposeBackward1>)
torch.Size([1, 10, 8])


In [11]:
cell

RNN(5, 8, batch_first=True)

In [12]:
print(_status)
print(_status.shape) # 최종 time-step의 hidden_state

tensor([[[nan, nan, nan, nan, nan, nan, nan, nan]]], grad_fn=<StackBackward0>)
torch.Size([1, 1, 8])


# 3. Deep RNN (깊은 순환신경망)

In [13]:
# num_layers 값을 늘려서 층을 쌓자
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

cell = nn.RNN(input_size=5, hidden_size=8, num_layers=2, batch_first=True)
outputs, _status = cell(inputs)
print(outputs.shape) # 모든 time-step의 hidden_state

torch.Size([1, 10, 8])


outputs shape은 동일하나 마지막 시점의 hidden_state가 달라진다:

In [14]:
cell

RNN(5, 8, num_layers=2, batch_first=True)

In [15]:
# (num_layers, batch_size, hidden_size)
print(_status.shape)

torch.Size([2, 1, 8])


# 4. Bidirectional RNN (양방향 순환신경망)

기본적으로 두 개의 메모리 셀을 사용

첫번째 메모리 셀은 앞 시점의 은닉 상태(Forward States)를 전달받아 현재의 은닉 상태를 계산한다.

두번째 메모리 셀은 앞 시점의 은닉 상태가 아니라 뒤 시점의 은닉 상태(Backward States)를 전달 받아 현재의 은닉 상태를 계산한다.

그리고 이 두 개의 값 모두가 출력층에서 출력값을 예측하기 위해 사용된다.

In [16]:
inputs = torch.Tensor(1, 10, 5)

# bidirectional=True 로 설정하면 양방향 RNN이 된다.
cell = nn.RNN(input_size=5, hidden_size=8, num_layers=2, batch_first=True, bidirectional=True)
outputs, _status = cell(inputs)

print(outputs.shape) # (batch_size, time_steps, hidden_size * 2) 크기를 가진다

torch.Size([1, 10, 16])


In [17]:
print(cell)

RNN(5, 8, num_layers=2, batch_first=True, bidirectional=True)


In [18]:
print(_status)
print(_status.shape) # (num_layers * 2, batch_size, hidden_size) 크기

tensor([[[-1.0000,  1.0000,  1.0000,  1.0000, -1.0000,  1.0000,  1.0000,
           1.0000]],

        [[ 1.0000,  1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000]],

        [[-0.5398, -0.7061,  0.8735, -0.1168, -0.9468,  0.1511, -0.8848,
          -0.8242]],

        [[-0.8775,  0.8700,  0.9854, -0.4266, -0.1077, -0.3785,  0.5749,
          -0.0163]]], grad_fn=<StackBackward0>)
torch.Size([4, 1, 8])
